In [1]:
import os
import sys
import configparser
from itertools import combinations
import pandas as pd
import numpy as np
import cv2
import itertools
from itertools import cycle
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report,ConfusionMatrixDisplay,RocCurveDisplay,roc_auc_score,roc_curve,auc,precision_recall_fscore_support,accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader,WeightedRandomSampler
from torch.utils.data import Dataset as BaseDataset

In [2]:
def create_dataframe(metadata_file):
    df = pd.read_csv(metadata_file)

    # encode labels
    df['class'] = df['class'].astype('category')
    df['label_encoded'] = df['class'].cat.codes.astype('int64')

    return df

In [3]:
def get_weak_transforms(img_size,img_mean,img_std):
    weak_transforms = A.Compose([
                        A.Resize(img_size, img_size),
                        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=10, shift_limit=0.1, p=1, border_mode=0),
                        A.Normalize(mean=img_mean, std=img_std),
                        ToTensorV2(),
                    ],additional_targets={'roi':'image'})
    return weak_transforms

def get_strong_transforms(img_size,img_mean,img_std):
    strong_transforms = A.Compose([
                        A.Resize(img_size, img_size),
                        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=10, shift_limit=0.1, p=1, border_mode=0),
                        A.OneOf(
                            [
                                A.CLAHE(p=1),
                                A.RandomBrightnessContrast(p=1),
                                A.RandomGamma(p=1),
                            ],
                            p=0.9,
                        ),
                        A.OneOf(
                            [
                                A.Sharpen(p=1),
                                A.Blur(blur_limit=3, p=1),
                                A.MotionBlur(blur_limit=3, p=1),
                            ],
                            p=0.9,
                        ),
                        A.OneOf(
                            [
                                A.RandomBrightnessContrast(p=1),
                                A.HueSaturationValue(p=1),
                            ],
                            p=0.9,
                        ),
                        A.Normalize(mean=img_mean, std=img_std),
                        ToTensorV2(),
                    ],additional_targets={'roi':'image'})
    return strong_transforms

def get_valid_transforms(img_size,img_mean,img_std):
    valid_transforms = A.Compose([
                        A.Resize(img_size, img_size),
                        A.Normalize(mean=img_mean, std=img_std),
                        ToTensorV2(),
                    ],additional_targets={'roi':'image'})
    return valid_transforms

def get_resize_transforms(img_size,img_mean,img_std):
    resize_transforms = A.Compose([
                        A.Resize(img_size, img_size),
                        A.Normalize(mean=img_mean, std=img_std),
                        ToTensorV2(),
                    ],additional_targets={'roi':'image'})
    return resize_transforms

In [4]:
class Dataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations."""
    
    def __init__(
            self,
            df,
            raw_dir,
            roi_dir,
            augmentation=None,
            visualize = False
    ):
        self.df = df.reset_index(drop=True)
        self.ids = self.df.loc[:,'file_name'].values
        self.images_fps = [os.path.join(raw_dir, image_id) for image_id in self.ids]
        self.roi_fps = [os.path.join(roi_dir, image_id) for image_id in self.ids]
        
        self.augmentation = augmentation
        self.visualize = visualize

        self.labels = sorted(self.df['label_encoded'].unique().tolist())
        self.label_names = self.df['class'].cat.categories
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        roi = cv2.imread(self.roi_fps[i])
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

        label = self.df.loc[i,'label_encoded']
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, roi=roi)
            image, roi = sample['image'], sample['roi']

        
        # Revert Normalize to visualize the image
            if self.visualize:
                invTrans = A.Normalize(mean=[-x/y for x,y in zip(img_mean,img_std)],
                                       std=[1/x for x in img_std],
                                       max_pixel_value=1.0,
                                       always_apply=True)
                image = image.detach().cpu().numpy().transpose(1,2,0)
                image = invTrans(image=image)['image']
                image = (image*255).astype(np.uint8)
        
        return image, roi, label
        
    def __len__(self):
        return len(self.ids)

In [5]:
class VisionModel:
    def __init__(self,
                 model_architecture,
                 num_classes,
                 dropout_percent=0.5,
                 pretrained=True,
                 device='cuda',
                 checkpoint=False,
                 eval_mode=True):
        self.model_architecture = model_architecture
        self.num_classes = num_classes
        self.dropout_percent = dropout_percent
        self.pretrained = pretrained
        self.device = torch.device(device)
        self.checkpoint=checkpoint
        self.eval_mode = eval_mode
        
        self._load_model()

    def _load_model(self):
        if self.model_architecture == 'convnext_small' or self.model_architecture == 'convnext_tiny':
            self.model = timm.create_model(self.model_architecture, pretrained=self.pretrained, num_classes=self.num_classes,drop_rate=self.dropout_percent)

        if self.model_architecture == 'efficientnet_b0' or self.model_architecture == 'efficientnet_b1':
            self.model = timm.create_model(self.model_architecture, pretrained=self.pretrained, num_classes=self.num_classes)
            num_ftrs = self.model.get_classifier().in_features
            if self.dropout_percent:
                self.model.classifier = nn.Sequential(
                                        nn.Dropout(self.dropout_percent),
                                        nn.Linear(num_ftrs,self.num_classes)
                )
            else:
                self.model.classifier = nn.Linear(num_ftrs, self.num_classes)

        if self.model_architecture == 'resnet50':
            self.model = timm.create_model(self.model_architecture, pretrained=self.pretrained, num_classes=self.num_classes)
            num_ftrs = self.model.get_classifier().in_features
            if self.dropout_percent:
                self.model.fc = nn.Sequential(
                                        nn.Dropout(self.dropout_percent),
                                        nn.Linear(num_ftrs,self.num_classes)
                )
            else:
                self.model.fc = nn.Linear(num_ftrs, self.num_classes)


        if self.checkpoint:
            self.model.load_state_dict(torch.load(self.checkpoint))
        if self.eval_mode:
            self.model = self.model.eval()
        
        self.model = self.model.to(self.device)   
        
    def __call__(self):
        return self.model

In [6]:
class Evaluate:
    def __init__(self,model,dataset,teacher=False):
        self.model = model()
        self.model_class = model
        self.dataset = dataset
        self.teacher = teacher
        self.class_ids = self.dataset.labels
        self.class_names = self.dataset.label_names
        self.label_array = np.array([], dtype='int64')
        self.prediction_array = np.array([], dtype='int64')
        self.softmax_array = np.array([], dtype='float64')
        
        self._perform_cls()
        self._fpr_tpr_calc()
        
    def _perform_cls(self):
        with torch.no_grad():
            self.model.eval()
            for self.images,self.rois, self.labels in self.dataset:
                if self.teacher:
                    self.images = self.rois.to(self.model_class.device).unsqueeze(0)
                else:
                    self.images = self.images.to(self.model_class.device).unsqueeze(0)
                #labels = labels.to(device)
                self.cls_outputs = self.model(self.images)
                self.softmax_array = np.append(self.softmax_array,F.softmax(self.cls_outputs,dim=1).detach().cpu().numpy())
                self.softmax_array = np.reshape(self.softmax_array,(-1,self.model.num_classes))
                _, self.predicted = torch.max(self.cls_outputs.data, 1)
                #_,labels = torch.max(labels.data, 0)
                self.label_array = np.append(self.label_array, self.labels)
                self.prediction_array = np.append(self.prediction_array, self.predicted.detach().cpu().numpy())
        
        self.label_array_oh = np.zeros((self.label_array.size, self.label_array.max()+1), dtype=int)
        self.label_array_oh[np.arange(self.label_array.size),self.label_array] = 1
    
    @property
    def print_cls_report(self):
        return classification_report(self.label_array, self.prediction_array, digits=4,zero_division=0)
    
    @property
    def cls_report(self):
        pre_rec_fs_sups = dict()
        
        for label in ['micro','macro','weighted']:
            prfs = precision_recall_fscore_support(
                self.label_array,
                self.prediction_array,
                average=label,
                zero_division=0
            )
            
            
            pre_rec_fs_sups[label] = {'precision' : prfs[0],
                                      'recall' : prfs[1],
                                      'f1-score' : prfs[2],
                                      'support' : prfs[3]
                                     }
        
        per_label_prfs = precision_recall_fscore_support(
            self.label_array,
            self.prediction_array,
            average=None,
            zero_division=0
        )
        
        for class_id in self.class_ids:
                pre_rec_fs_sups[class_id] = {'precision':per_label_prfs[0][class_id],
                                             'recall': per_label_prfs[1][class_id],
                                             'f1-score' : per_label_prfs[2][class_id],
                                             'support' : per_label_prfs[3][class_id]
                                            }
                
        # for class_id,class_name in zip(self.class_ids,self.class_names):
        #         pre_rec_fs_sups[class_name] = {'precision':per_label_prfs[0][class_id],
        #                                      'recall': per_label_prfs[1][class_id],
        #                                      'f1-score' : per_label_prfs[2][class_id],
        #                                      'support' : per_label_prfs[3][class_id]
        #                                     }
        
        acc = accuracy_score(self.label_array,self.prediction_array)
        pre_rec_fs_sups['accuracy'] = acc
        
        return pre_rec_fs_sups
    
    @property
    def auc(self):
        aucs = dict()
        for label in ['micro','macro','weighted']:
            aucs[label] = roc_auc_score(
                self.label_array_oh,
                self.softmax_array,
                multi_class="ovr",
                average=label,
            )
        
        aucs['macro_ovo'] = roc_auc_score(
                self.label_array_oh,
                self.softmax_array,
                multi_class="ovo",
                average='macro',
        )
        
        per_label_auc = roc_auc_score(
            self.label_array_oh,
            self.softmax_array,
            multi_class="ovr",
            average=None,
        )   
        
        for i,value in enumerate(per_label_auc):
            aucs[f'{i}'] = value

        return aucs
    
    def _fpr_tpr_calc(self):
        fpr_grid = np.linspace(0.0, 1.0, 1000)
        # store the fpr, tpr, and roc_auc for all averaging strategies
        self.fpr, self.tpr, self.roc_auc = dict(), dict(), dict()
        # Compute micro-average ROC curve and ROC area for OvR strategy
        self.fpr["micro"], self.tpr["micro"], _ = roc_curve(self.label_array_oh.ravel(), self.softmax_array.ravel(),drop_intermediate=True)
        self.roc_auc["micro"] = auc(self.fpr["micro"], self.tpr["micro"])
        
        interp_tpr = np.interp(fpr_grid,self.fpr['micro'],self.tpr['micro'])
        interp_tpr[0] = 0.0
        self.fpr['micro'] = fpr_grid
        self.tpr['micro'] = interp_tpr
        
        
        self.tresholds = dict()
        for i in range(self.model.num_classes):
            self.fpr[i], self.tpr[i], self.tresholds[i] = roc_curve(self.label_array_oh[:, i], self.softmax_array[:, i])
            self.roc_auc[i] = auc(self.fpr[i], self.tpr[i])

        # Interpolate all ROC curves at these points
        mean_tpr = np.zeros_like(fpr_grid)

        for i in range(self.model.num_classes):
            mean_tpr += np.interp(fpr_grid, self.fpr[i], self.tpr[i])  # linear interpolation

        # Average it and compute AUC
        mean_tpr /= self.model.num_classes

        self.fpr["macro"] = fpr_grid
        self.tpr["macro"] = mean_tpr
        self.roc_auc["macro"] = auc(self.fpr["macro"], self.tpr["macro"])
        
        # Compute macro-average ROC curve and ROC area for OvO strategy
        pair_list = list(combinations(self.class_ids,2))
        
        pair_scores = []
        self.tpr['macro_ovo'] = np.zeros_like(fpr_grid)
        mean_tpr = dict()
        for ix, (label_a, label_b) in enumerate(pair_list):

            a_mask = self.label_array == label_a
            b_mask = self.label_array == label_b
            ab_mask = np.logical_or(a_mask, b_mask)

            a_true = a_mask[ab_mask]
            b_true = b_mask[ab_mask]

            idx_a = np.flatnonzero(np.array(self.class_ids) == label_a)[0]
            idx_b = np.flatnonzero(np.array(self.class_ids) == label_b)[0]

            fpr_a, tpr_a, _ = roc_curve(a_true, self.softmax_array[ab_mask, idx_a])
            fpr_b, tpr_b, _ = roc_curve(b_true, self.softmax_array[ab_mask, idx_b])

            mean_tpr[ix] = np.zeros_like(fpr_grid)
            mean_tpr[ix] += np.interp(fpr_grid, fpr_a, tpr_a)
            mean_tpr[ix] += np.interp(fpr_grid, fpr_b, tpr_b)
            mean_tpr[ix] /= 2
            mean_score = auc(fpr_grid, mean_tpr[ix])
            pair_scores.append(mean_score)
            self.tpr['macro_ovo'] += mean_tpr[ix]
            
        self.tpr['macro_ovo'] /= sum(1 for pair in enumerate(pair_list))
        self.fpr['macro_ovo'] = fpr_grid
        self.roc_auc['macro_ovo'] = np.average(pair_scores)
    
    def roc(self,contents=[]):
        assert contents is not None
        fig, ax = plt.subplots(figsize=(6, 6))
        
        colors = cycle(["tab:blue","tab:orange","tab:green","tab:red","tab:purple","tab:brown"])
        
        for class_id, color in zip([x for x in contents if x not in ['micro','macro']], colors):
            plt.plot(
                self.fpr[class_id],
                self.tpr[class_id],
                label=f"ROC curve for {class_id} (AUC = {self.roc_auc[class_id]:.2f})",
                color=color     
            )
        
        if 'micro' in contents:
            plt.plot(
                self.fpr["micro"],
                self.tpr["micro"],
                label=f"micro-average ROC curve (AUC = {self.roc_auc['micro']:.2f})",
                color="deeppink",
                linestyle=":",
                linewidth=4,
            )
        
        if 'macro' in contents:
            plt.plot(
                self.fpr["macro"],
                self.tpr["macro"],
                label=f"macro-average ROC curve (AUC = {self.roc_auc['macro']:.2f})",
                color="navy",
                linestyle=":",
                linewidth=4,
            )
            
        plt.plot([0, 1], [0, 1], "k--")
        plt.axis("square")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic")
        plt.legend()
        return fig,ax

In [7]:
def cv_cls_report(ev_models,labels,folds,digits=4):
    cv_cls_report = {}
    labels.extend(['micro','macro', 'weighted'])
    folds = range(folds)
    accs = []
    for fold in folds:
        accs.append(ev_models[fold].cls_report['accuracy'])

    cv_cls_report['accuracy'] = [round(np.mean(accs),digits),round(np.std(accs),digits)]

    avg_prec = {}
    avg_rec = {}
    avg_f1 = {}
    for label in labels:
        precs = []
        recs = []
        f1s = []
        sups = []
        for fold in folds:
            precs.append(ev_models[fold].cls_report[label]['precision'])
            recs.append(ev_models[fold].cls_report[label]['recall'])
            f1s.append(ev_models[fold].cls_report[label]['f1-score'])
            
        avg_prec[label] = [round(np.mean(precs),digits),round(np.std(precs),digits)]
        avg_rec[label] = [round(np.mean(recs),digits),round(np.std(recs),digits)]
        avg_f1[label] = [round(np.mean(f1s),digits),round(np.std(f1s),digits)]

        metric_dict = {}
        metric_dict['precision'] = avg_prec[label]
        metric_dict['recall'] = avg_rec[label]
        metric_dict['f1-score'] = avg_f1[label]
        cv_cls_report[label] = metric_dict
        
    return cv_cls_report

In [8]:
def cv_auc(ev_models,contents,n_folds,digits=4):
    avg_auc = {}
    for class_id in contents:
        values = []
        for fold in range(n_folds):
            if class_id == 'weighted':
                values.append(ev_models[fold].auc[class_id])
            else:
                values.append(ev_models[fold].roc_auc[class_id])
        avg_auc[class_id] = [round(np.mean(values),digits),round(np.std(values),digits)]
    return avg_auc

In [9]:
def cv_roc(ev_models,contents,folds,ax,title='Receiver Operating Characteristic',save=None):
    avg_tpr = {}
    avg_fpr = {}
    avg_auc = {}
    std_auc = {}

    colors = cycle(["tab:blue","tab:orange","tab:green","tab:red","tab:purple","tab:brown"])

    for class_id, color in zip([x for x in contents if x not in ['micro','macro','macro_ovo']], colors):
        fpr_grid = np.linspace(0.0, 1.0, 1000)
        mean_tpr = np.zeros_like(fpr_grid)
        aucs = []
        for fold in range(folds):
            mean_tpr += np.interp(fpr_grid, ev_models[fold].fpr[class_id], ev_models[fold].tpr[class_id])
            aucs.append(ev_models[fold].roc_auc[class_id])

        avg_tpr[class_id] = mean_tpr / folds
        avg_fpr[class_id] = fpr_grid
        avg_auc[class_id] = np.mean(aucs)
        std_auc[class_id] = np.std(aucs)
        
        ax.plot(
            avg_fpr[class_id],
            avg_tpr[class_id],
            label=f"Severity {class_id} (AUC = {avg_auc[class_id]:.2f} $\pm$ {std_auc[class_id]:.2f})" if class_id != 0 else f"Healthy (AUC = {avg_auc[class_id]:.2f} $\pm$ {std_auc[class_id]:.2f})",
            color=color     
        )
        
    if 'micro' in contents:
        tprs = []
        fprs = []
        aucs = []
        for fold in range(folds):
            tprs.append(ev_models[fold].tpr['micro'])
            fprs.append(ev_models[fold].fpr['micro'])
            aucs.append(ev_models[fold].roc_auc['micro'])
        avg_fpr["micro"] = np.mean(fprs,axis=0)
        avg_tpr["micro"] = np.mean(tprs,axis=0)
        avg_auc["micro"] = np.mean(aucs)
        std_auc["micro"] = np.std(aucs)
         
        ax.plot(
            avg_fpr["micro"],
            avg_tpr["micro"],
            label=f"micro-average (AUC = {avg_auc['micro']:.2f} $\pm$ {std_auc['micro']:.2f})",
            color="deeppink",
            linestyle=":",
            linewidth=4,
        )
        
    if 'macro' in contents:
        tprs = []
        fprs = []
        aucs = []
        for fold in range(folds):
            tprs.append(ev_models[fold].tpr['macro'])
            fprs.append(ev_models[fold].fpr['macro'])
            aucs.append(ev_models[fold].roc_auc['macro'])
        avg_fpr["macro"] = np.mean(fprs,axis=0)
        avg_tpr["macro"] = np.mean(tprs,axis=0)
        avg_auc["macro"] = np.mean(aucs)
        std_auc["macro"] = np.std(aucs)
            
        ax.plot(
            avg_fpr["macro"],
            avg_tpr["macro"],
            label=f"macro-average (AUC = {avg_auc['macro']:.2f} $\pm$ {std_auc['macro']:.2f})",
            color="navy",
            linestyle=":",
            linewidth=4,
        )
        
    if 'macro_ovo' in contents:
        tprs = []
        fprs = []
        aucs = []
        for fold in range(folds):
            tprs.append(ev_models[fold].tpr['macro_ovo'])
            fprs.append(ev_models[fold].fpr['macro_ovo'])
            aucs.append(ev_models[fold].roc_auc['macro_ovo'])
        avg_fpr["macro_ovo"] = np.mean(fprs,axis=0)
        avg_tpr["macro_ovo"] = np.mean(tprs,axis=0)
        avg_auc["macro_ovo"] = np.mean(aucs)
        std_auc["macro_ovo"] = np.std(aucs)
            
        ax.plot(
            avg_fpr["macro"],
            avg_tpr["macro"],
            label=f"OvO macro-average (AUC = {avg_auc['macro_ovo']:.2f} $\pm$ {std_auc['macro_ovo']:.2f})",
            color="cyan",
            linestyle=":",
            linewidth=4,
        )

    ax.plot([0, 1], [0, 1], "k--")
    ax.axis("square")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(title)
    ax.legend()

In [10]:
def load_experiment(name):
    config_file = f'{name}_config.txt'
    config = configparser.ConfigParser()
    config.read(config_file)

    model_dir = config['settings']['model_dir']
    repeated_runs = int(config['settings']['repeated_runs'])

    if 'model' in config:
        dropout_percent = float(config['model']['dropout_percent'])
        model_architecture = config['model']['model_architecture']
    else:
        dropout_percent = (float(config['teacher']['dropout_percent']),float(config['student']['dropout_percent']))
        model_architecture = (config['teacher']['model_architecture'],config['student']['model_architecture'])

    return model_dir, model_architecture, dropout_percent, repeated_runs

In [11]:
def plot_loss_curve(experiment_name,model_architecture,model_type,plot_type,repeated_runs,ax,student_losses=[]):
    if model_type == 'baseline':
        logs = pd.read_csv(f'logs/{experiment_name}_{model_architecture}_metrics.csv')
    elif model_type=='teacher':
        logs = pd.read_csv(f'logs/{experiment_name}_teacher_{model_architecture}_metrics.csv')
    elif model_type=='student':
        logs = pd.read_csv(f'logs/{experiment_name}_student_{model_architecture}_metrics.csv')

    train_values = []
    valid_values = []
    train_cls_values = []
    train_cons_values = []
    for fold in range(repeated_runs):
        train_values.append(logs[f'fold{fold}_train_{plot_type}'].values)
        if model_type=='student':
            train_cls_values.append(logs[f'fold{fold}_train_cls_loss'].values)
            train_cons_values.append(logs[f'fold{fold}_train_cons_loss'].values)
        valid_values.append(logs[f'fold{fold}_val_{plot_type}'].values)
    
    mean_train_values = np.mean(train_values,axis=0)
    mean_valid_values = np.mean(valid_values,axis=0)

    if model_type=='student':
        mean_train_cls_values = np.mean(train_cls_values,axis=0)
        mean_train_cons_values = np.mean(train_cons_values,axis=0)
    
    for values in train_values:
        ax.plot(range(len(logs)),values,color="tab:blue",linewidth=1,alpha=0.3)

    if 'cons' in student_losses and plot_type=='loss':
        for values in train_cons_values:
            ax.plot(range(len(logs)),values,color="tab:green",linewidth=1,alpha=0.3)

        ax.plot(range(len(logs)),mean_train_cons_values,label=f'Train MSE {plot_type}',color="tab:green",linewidth=3)

    if 'cls' in student_losses and plot_type=='loss':
        for values in train_cls_values:
            ax.plot(range(len(logs)),values,color="tab:purple",linewidth=1,alpha=0.3)

        ax.plot(range(len(logs)),mean_train_cls_values,label=f'Train CE {plot_type}',color="tab:purple",linewidth=3)
        
    # Plot mean
    if model_type=='student' and plot_type=='loss':
        label = 'Train loss combined'
    else:
        label = f'Train {plot_type}'
        
    ax.plot(range(len(logs)),mean_train_values,label=label,color="tab:blue",linewidth=3)

    for values in valid_values:
        ax.plot(range(len(logs)),values,color="tab:orange",linewidth=1,alpha=0.3)
        
    # Plot mean
    ax.plot(range(len(logs)),mean_valid_values,label=f'Valid {plot_type}',color="tab:orange",linewidth=3)

    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy' if plot_type=='acc' else 'Loss')
    ax.legend()

In [12]:
# Init data
raw_data_dir = '../../../data/segmentation_test/raw'
roi_data_dir = '../../../data/segmentation_test/roi'
metadata_file = '../../../data/segmentation_test/image_patient_splits.csv'
resolution = 224
img_mean = IMAGENET_DEFAULT_MEAN
img_std = IMAGENET_DEFAULT_STD

df = create_dataframe(metadata_file)

num_classes = 5

valid_dataset = Dataset(
    df[df['split']=='valid'],
    raw_dir=raw_data_dir,
    roi_dir=roi_data_dir,
    augmentation=get_valid_transforms(resolution,img_mean,img_std), 
)

vis_dataset = Dataset(
    df, 
    raw_dir=raw_data_dir,
    roi_dir=roi_data_dir,
    augmentation=get_resize_transforms(800,img_mean,img_std), 
    visualize=True
)

#### Evaluation

In [13]:
def get_metrics(ev_models,repeated_runs):
    metrics_list = []
    rep = cv_cls_report(ev_models,[0,1,2,3,4],repeated_runs)
    for label in [0,1,2,3,4]:
        for metric in ['precision','recall','f1-score']:
            for value in [0,1]:
                metrics_list.append(rep[label][metric][value])

    acc_mean = rep['accuracy'][0]
    acc_std = rep['accuracy'][1]

    return metrics_list,acc_mean,acc_std

In [14]:
def calc_save_metrics(experiment_names,csv_name):
    vm_list = []
    ev_list = []
    metrics_dict_all = {}
    acc_mean_all = []
    acc_std_all = []

    for experiment in experiment_names:
        model_dir, model_architecture, dropout_percent, repeated_runs = load_experiment(experiment)

        vision_models = {}
        ev_models = {}
        vision_models_teacher = {}
        ev_models_teacher = {}
        vision_models_student = {}
        ev_models_student = {}

        if 'Baseline' in experiment:
            for fold in range(repeated_runs):
                vision_models[fold] = VisionModel(model_architecture,
                                            num_classes=5,
                                            dropout_percent=dropout_percent,
                                            checkpoint=f'./{model_dir}/{experiment}_{model_architecture}_run_{fold}.pth')
            
                ev_models[fold] = Evaluate(vision_models[fold],valid_dataset)

            vm_list.append(vision_models)
            ev_list.append(ev_models)

            metrics_list, acc_mean, acc_std = get_metrics(ev_models,repeated_runs)
            metrics_dict_all[experiment] = metrics_list
            acc_mean_all.append(acc_mean)
            acc_std_all.append(acc_std)

            fig, axs = plt.subplots(1,2,figsize=(10,4),dpi=150)
            plot_loss_curve(experiment,model_architecture,model_type='baseline',plot_type='acc',repeated_runs=repeated_runs,ax=axs[0])
            plot_loss_curve(experiment,model_architecture,model_type='baseline',plot_type='loss',repeated_runs=repeated_runs,ax=axs[1])
            plt.savefig(f'export/{experiment}_acc_loss_curve.png')
            plt.close()
        
        if 'KD' in experiment:
            for fold in range(repeated_runs):
                vision_models_teacher[fold] = VisionModel(model_architecture[0],
                                            num_classes=5,
                                            dropout_percent=dropout_percent[0],
                                            checkpoint=f'./{model_dir}/{experiment}_teacher_{model_architecture[0]}_run_{fold}.pth')
            
                ev_models_teacher[fold] = Evaluate(vision_models_teacher[fold],valid_dataset,teacher=True)

                vision_models_student[fold] = VisionModel(model_architecture[1],
                                                num_classes=5,
                                                dropout_percent=dropout_percent[1],
                                                checkpoint=f'./{model_dir}/{experiment}_student_{model_architecture[1]}_run_{fold}.pth')
                
                ev_models_student[fold] = Evaluate(vision_models_student[fold],valid_dataset)

            vm_list.append([ev_models_teacher,ev_models_student])
            ev_list.append([ev_models_teacher,ev_models_student])

            metrics_list, acc_mean, acc_std = get_metrics(ev_models_teacher,repeated_runs)
            metrics_dict_all[f'{experiment}_teacher'] = metrics_list
            acc_mean_all.append(acc_mean)
            acc_std_all.append(acc_std)

            metrics_list, acc_mean, acc_std = get_metrics(ev_models_student,repeated_runs)
            metrics_dict_all[f'{experiment}_student'] = metrics_list
            acc_mean_all.append(acc_mean)
            acc_std_all.append(acc_std)

            fig, axs = plt.subplots(1,2,figsize=(10,4),dpi=150)
            plot_loss_curve(experiment,model_architecture[0],model_type='teacher',plot_type='acc',repeated_runs=repeated_runs,ax=axs[0])
            plot_loss_curve(experiment,model_architecture[0],model_type='teacher',plot_type='loss',repeated_runs=repeated_runs,ax=axs[1])
            plt.savefig(f'export/{experiment}_teacher_acc_loss_curve.png')
            plt.close()

            fig, axs = plt.subplots(1,2,figsize=(10,4),dpi=150)
            plot_loss_curve(experiment,model_architecture[1],model_type='student',plot_type='acc',repeated_runs=repeated_runs,ax=axs[0])
            plot_loss_curve(experiment,model_architecture[1],model_type='student',plot_type='loss',repeated_runs=repeated_runs,ax=axs[1],student_losses=['cons','cls'])
            plt.savefig(f'export/{experiment}_student_acc_loss_curve.png')
            plt.close()

    df = pd.DataFrame.from_dict(metrics_dict_all,orient='index')
    df.columns = pd.MultiIndex.from_product([[0,1,2,3,4],['Prec','Rec','F1'],['mean','std']])
    df['acc_mean'] = acc_mean_all
    df['acc_std'] = acc_std_all

    df.to_csv(f'export/{csv_name}')

In [15]:
def print_metric_for_latex(df,model_index,metric_index):
    print(df.transpose().iloc[0,model_index])
    f = ''
    for i,value in enumerate(df.transpose().iloc[metric_index,model_index].values):
        if (i % 2) == 0:
            f = f+str(value)+' $\\pm$ '
        else:
            f = f+str(value)+' & '
    print(f)

In [16]:
def print_eval_table_latex(df,experiment_names):
    text = ''
    for i,cls in enumerate(['Bacterial','COVID-19','Healthy','Fungal','Viral (other)']):
        text = text + cls + ' & '
        for k,(metric,model,stat) in enumerate(itertools.product(['Prec','Rec','F1'],experiment_names,['mean','std'])):
            if (k % 2) == 0:
                text = text + df.loc[df['model']==model,f'{i}_{metric}_{stat}'].item()[1:]+' $\\pm$ '
            else:
                text = text + df.loc[df['model']==model,f'{i}_{metric}_{stat}'].item()[1:]+' & '
        text = text[:-2] + '\\\\' +'\n'
    print(text)

#### Dropout Experiments

In [17]:
valid_dataset.labels

[0, 1, 2, 3, 4]

In [18]:
valid_dataset.label_names

Index(['B', 'C', 'NB', 'P', 'V'], dtype='object')

In [19]:
precision_index = [1,2,7,8,13,14,19,20,25,26]
recall_index = [3,4,9,10,15,16,21,22,27,28]

In [20]:
#experiment_names = ['ConvNeXt_small_Baseline','ConvNeXt_tiny_Baseline','EB0_Baseline','EB1_Baseline','ResNet50_Baseline','ConvNeXt_small_KD','ConvNeXt_tiny_KD','EB0_KD','EB1_KD','ResNet50_KD']
#calc_save_metrics(experiment_names,csv_name='model_metrics_do.csv')

In [21]:
df = pd.read_csv('export/model_metrics_do.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_Baseline,0.326,0.0498,0.3474,0.0976,0.3307,0.0579,0.8292,0.0573,0.7478,...,0.5179,0.0386,0.1329,0.1097,0.1,0.0848,0.1136,0.0949,0.7853,0.0037
3,ConvNeXt_tiny_Baseline,0.3534,0.0405,0.3579,0.1021,0.3515,0.0707,0.7218,0.0693,0.8,...,0.3811,0.1745,0.3369,0.0778,0.3125,0.1854,0.2903,0.0981,0.7825,0.0147
4,EB0_Baseline,0.2952,0.1805,0.1474,0.1021,0.1959,0.1295,0.6999,0.0435,0.7739,...,0.5902,0.0637,0.1152,0.1029,0.1,0.0935,0.1069,0.0977,0.7871,0.0122
5,EB1_Baseline,0.2273,0.1211,0.1158,0.0698,0.1526,0.0878,0.665,0.0782,0.7652,...,0.4871,0.065,0.3607,0.0746,0.425,0.1075,0.3842,0.0738,0.7760,0.0119
6,ResNet50_Baseline,0.3107,0.2539,0.0947,0.0614,0.1428,0.095,0.7188,0.1223,0.7043,...,0.5725,0.0574,0.0545,0.1091,0.075,0.15,0.0632,0.1263,0.7825,0.0114
7,ConvNeXt_small_KD_teacher,0.437,0.0567,0.5789,0.2642,0.4742,0.1659,0.8601,0.0505,0.8783,...,0.6045,0.0627,0.2,0.1778,0.0875,0.0935,0.1171,0.1188,0.8369,0.0164
8,ConvNeXt_small_KD_student,0.365,0.0328,0.3895,0.0714,0.3734,0.0376,0.7906,0.0692,0.7826,...,0.581,0.0294,0.2167,0.1944,0.1125,0.1212,0.1449,0.1487,0.8083,0.0103
9,ConvNeXt_tiny_KD_teacher,0.4168,0.0861,0.4947,0.1182,0.4507,0.0973,0.8785,0.0687,0.8087,...,0.5081,0.063,0.2951,0.0388,0.3125,0.1046,0.2976,0.0701,0.8157,0.0120
10,ConvNeXt_tiny_KD_student,0.2982,0.0191,0.2737,0.0394,0.2828,0.0185,0.7875,0.0472,0.7826,...,0.5853,0.0741,0.0862,0.106,0.0625,0.0791,0.0721,0.0899,0.7935,0.0147
11,EB0_KD_teacher,0.4259,0.0629,0.4632,0.1389,0.4392,0.1028,0.8423,0.0488,0.8174,...,0.6533,0.0209,0.1333,0.1633,0.05,0.0729,0.0691,0.0947,0.8332,0.0122


In [22]:
print_eval_table_latex(df,['ConvNeXt_small_Baseline','ConvNeXt_small_KD_student'])

Bacterial & .326 $\pm$ .0498 & .365 $\pm$ .0328 & .3474 $\pm$ .0976 & .3895 $\pm$ .0714 & .3307 $\pm$ .0579 & .3734 $\pm$ .0376 \\
COVID-19 & .8292 $\pm$ .0573 & .7906 $\pm$ .0692 & .7478 $\pm$ .0928 & .7826 $\pm$ .0615 & .7793 $\pm$ .0296 & .7864 $\pm$ .0644 \\
Healthy & .9641 $\pm$ .0028 & .9745 $\pm$ .0074 & .9923 $\pm$ .0049 & .9954 $\pm$ .0038 & .978 $\pm$ .0016 & .9848 $\pm$ .0033 \\
Fungal & .4946 $\pm$ .0141 & .5282 $\pm$ .0185 & .5517 $\pm$ .0899 & .6483 $\pm$ .0593 & .5179 $\pm$ .0386 & .581 $\pm$ .0294 \\
Viral (other) & .1329 $\pm$ .1097 & .2167 $\pm$ .1944 & .1 $\pm$ .0848 & .1125 $\pm$ .1212 & .1136 $\pm$ .0949 & .1449 $\pm$ .1487 \\



In [23]:
print_metric_for_latex(df,0,precision_index)

ConvNeXt_small_Baseline
0.326 $\pm$ 0.0498 & 0.8292 $\pm$ 0.0573 & 0.9641 $\pm$ 0.0028 & 0.4946 $\pm$ 0.0141 & 0.1329 $\pm$ 0.1097 & 


In [24]:
print_metric_for_latex(df,0,recall_index)

ConvNeXt_small_Baseline
0.3474 $\pm$ 0.0976 & 0.7478 $\pm$ 0.0928 & 0.9923 $\pm$ 0.0049 & 0.5517 $\pm$ 0.0899 & 0.1 $\pm$ 0.0848 & 


In [25]:
print_metric_for_latex(df,6,precision_index)

ConvNeXt_small_KD_student
0.365 $\pm$ 0.0328 & 0.7906 $\pm$ 0.0692 & 0.9745 $\pm$ 0.0074 & 0.5282 $\pm$ 0.0185 & 0.2167 $\pm$ 0.1944 & 


In [26]:
print_metric_for_latex(df,6,recall_index)

ConvNeXt_small_KD_student
0.3895 $\pm$ 0.0714 & 0.7826 $\pm$ 0.0615 & 0.9954 $\pm$ 0.0038 & 0.6483 $\pm$ 0.0593 & 0.1125 $\pm$ 0.1212 & 


#### No-Dropout Experiments

In [27]:
#experiment_names = ['ConvNeXt_small_Baseline_nodo','ConvNeXt_tiny_Baseline_nodo','EB0_Baseline_nodo','EB1_Baseline_nodo','ResNet50_Baseline_nodo',
#                    'ConvNeXt_small_KD_nodo','ConvNeXt_tiny_KD_nodo','EB0_KD_nodo','EB1_KD_nodo','ResNet50_KD_nodo']
#calc_save_metrics(experiment_names,csv_name='model_metrics_nodo.csv')

In [28]:
df = pd.read_csv('export/model_metrics_nodo.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_Baseline_nodo,0.4575,0.1736,0.2947,0.0714,0.3348,0.0202,0.7092,0.0812,0.8,...,0.567,0.0821,0.3312,0.2339,0.1375,0.1,0.1616,0.0964,0.7972,0.0157
3,ConvNeXt_tiny_Baseline_nodo,0.3067,0.0413,0.3158,0.0744,0.3079,0.0522,0.7736,0.0297,0.7304,...,0.3503,0.1554,0.2501,0.0959,0.3625,0.15,0.2909,0.111,0.7622,0.0159
4,EB0_Baseline_nodo,0.2619,0.1305,0.0842,0.0421,0.1242,0.0551,0.6398,0.0491,0.8348,...,0.3803,0.2299,0.1636,0.1447,0.275,0.2291,0.2038,0.175,0.7622,0.0280
5,EB1_Baseline_nodo,0.3615,0.0673,0.2,0.0906,0.2508,0.096,0.6665,0.0631,0.8174,...,0.4052,0.0514,0.2708,0.0647,0.375,0.1046,0.3129,0.0769,0.7668,0.0186
6,ResNet50_Baseline_nodo,0.3102,0.18,0.0947,0.0516,0.1428,0.077,0.6924,0.0648,0.7652,...,0.4972,0.0646,0.2185,0.1168,0.2375,0.1741,0.2123,0.1418,0.7733,0.0147
7,ConvNeXt_small_KD_nodo_teacher,0.4286,0.0729,0.4737,0.1412,0.4472,0.1016,0.8356,0.0708,0.8261,...,0.566,0.0866,0.2853,0.0326,0.2125,0.075,0.2319,0.0452,0.8212,0.0171
8,ConvNeXt_small_KD_nodo_student,0.3847,0.0535,0.4526,0.1272,0.4071,0.0643,0.8259,0.0705,0.7565,...,0.5546,0.1154,0.4125,0.3345,0.1375,0.1075,0.172,0.1071,0.8065,0.0154
9,ConvNeXt_tiny_KD_nodo_teacher,0.4829,0.0391,0.5684,0.1348,0.5154,0.0648,0.9051,0.0635,0.8348,...,0.5986,0.0847,0.2341,0.1345,0.15,0.0848,0.1783,0.0985,0.8350,0.0144
10,ConvNeXt_tiny_KD_nodo_student,0.3202,0.0665,0.2947,0.1031,0.3022,0.0743,0.7533,0.0321,0.8087,...,0.5532,0.0437,0.2352,0.1005,0.15,0.0848,0.1774,0.0865,0.7945,0.0095
11,EB0_KD_nodo_teacher,0.3775,0.0729,0.3368,0.1084,0.3526,0.0923,0.8292,0.0829,0.8348,...,0.589,0.0412,0.0983,0.0812,0.05,0.0468,0.0652,0.0575,0.8147,0.0018


In [29]:
print_metric_for_latex(df,0,precision_index)

ConvNeXt_small_Baseline_nodo
0.4575 $\pm$ 0.1736 & 0.7092 $\pm$ 0.0812 & 0.9474 $\pm$ 0.0079 & 0.5457 $\pm$ 0.0348 & 0.3312 $\pm$ 0.2339 & 


In [30]:
print_metric_for_latex(df,0,recall_index)

ConvNeXt_small_Baseline_nodo
0.2947 $\pm$ 0.0714 & 0.8 $\pm$ 0.1051 & 0.9954 $\pm$ 0.0062 & 0.6 $\pm$ 0.1337 & 0.1375 $\pm$ 0.1 & 


In [31]:
print_metric_for_latex(df,6,precision_index)

ConvNeXt_small_KD_nodo_student
0.3847 $\pm$ 0.0535 & 0.8259 $\pm$ 0.0705 & 0.9646 $\pm$ 0.0171 & 0.5333 $\pm$ 0.0538 & 0.4125 $\pm$ 0.3345 & 


In [32]:
print_metric_for_latex(df,6,recall_index)

ConvNeXt_small_KD_nodo_student
0.4526 $\pm$ 0.1272 & 0.7565 $\pm$ 0.0895 & 0.9954 $\pm$ 0.0062 & 0.6 $\pm$ 0.1858 & 0.1375 $\pm$ 0.1075 & 


#### Dropout+Weakaug Experiments

In [33]:
#experiment_names = ['ConvNeXt_small_Baseline_weakaug','ConvNeXt_tiny_Baseline_weakaug','EB0_Baseline_weakaug','EB1_Baseline_weakaug','ResNet50_Baseline_weakaug',
#                    'ConvNeXt_small_KD_weakaug','ConvNeXt_tiny_KD_weakaug','EB0_KD_weakaug','EB1_KD_weakaug','ResNet50_KD_weakaug']
#calc_save_metrics(experiment_names,csv_name='model_metrics_weakaug.csv')

In [34]:
df = pd.read_csv('export/model_metrics_weakaug.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_Baseline_weakaug,0.3629,0.07,0.3263,0.0394,0.3418,0.0464,0.7619,0.0952,0.8435,...,0.5092,0.052,0.3439,0.0426,0.275,0.0848,0.2935,0.0587,0.7963,0.0114
3,ConvNeXt_tiny_Baseline_weakaug,0.3114,0.0307,0.3684,0.0577,0.3341,0.023,0.8125,0.0873,0.7304,...,0.4792,0.0725,0.2738,0.0807,0.1875,0.1118,0.2112,0.1037,0.7816,0.0069
4,EB0_Baseline_weakaug,0.2913,0.1548,0.1579,0.1245,0.1979,0.1338,0.6402,0.0358,0.8261,...,0.5501,0.0814,0.1305,0.1134,0.0625,0.0559,0.0837,0.0737,0.7797,0.0107
5,EB1_Baseline_weakaug,0.3222,0.3855,0.0737,0.0788,0.1116,0.1156,0.538,0.0881,0.8783,...,0.481,0.064,0.3452,0.1049,0.325,0.1741,0.3231,0.139,0.7631,0.0211
6,ResNet50_Baseline_weakaug,0.2237,0.0982,0.0842,0.0421,0.1212,0.059,0.7265,0.035,0.6609,...,0.6031,0.0113,0.0,0.0,0.0,0.0,0.0,0.0,0.7797,0.0084
7,ConvNeXt_small_KD_weakaug_teacher,0.3851,0.0929,0.3263,0.1805,0.3448,0.1428,0.907,0.0436,0.8261,...,0.6054,0.0523,0.1821,0.1905,0.0875,0.075,0.1143,0.1048,0.8249,0.0065
8,ConvNeXt_small_KD_weakaug_student,0.3081,0.0713,0.2947,0.1582,0.2894,0.1178,0.7422,0.0594,0.7739,...,0.5162,0.1423,0.2017,0.1664,0.125,0.1425,0.1377,0.1341,0.7935,0.0144
9,ConvNeXt_tiny_KD_weakaug_teacher,0.4716,0.121,0.5158,0.161,0.491,0.1371,0.8713,0.0625,0.8435,...,0.5268,0.0974,0.2502,0.1295,0.3,0.1829,0.2656,0.1394,0.8258,0.0245
10,ConvNeXt_tiny_KD_weakaug_student,0.3359,0.031,0.3789,0.0614,0.3549,0.041,0.7881,0.0736,0.8,...,0.5707,0.0526,0.12,0.16,0.05,0.0612,0.0689,0.0851,0.8018,0.0105
11,EB0_KD_weakaug_teacher,0.4578,0.0474,0.4421,0.1315,0.4447,0.0919,0.865,0.0508,0.8696,...,0.5202,0.1076,0.2684,0.1083,0.25,0.1811,0.2345,0.127,0.8221,0.0115


In [35]:
print_metric_for_latex(df,0,precision_index)

ConvNeXt_small_Baseline_weakaug
0.3629 $\pm$ 0.07 & 0.7619 $\pm$ 0.0952 & 0.9598 $\pm$ 0.0073 & 0.537 $\pm$ 0.0238 & 0.3439 $\pm$ 0.0426 & 


In [36]:
print_metric_for_latex(df,0,recall_index)

ConvNeXt_small_Baseline_weakaug
0.3263 $\pm$ 0.0394 & 0.8435 $\pm$ 0.0651 & 0.9892 $\pm$ 0.0038 & 0.4897 $\pm$ 0.0828 & 0.275 $\pm$ 0.0848 & 


In [37]:
print_metric_for_latex(df,6,precision_index)

ConvNeXt_small_KD_weakaug_student
0.3081 $\pm$ 0.0713 & 0.7422 $\pm$ 0.0594 & 0.9758 $\pm$ 0.0087 & 0.4701 $\pm$ 0.0648 & 0.2017 $\pm$ 0.1664 & 


In [38]:
print_metric_for_latex(df,6,recall_index)

ConvNeXt_small_KD_weakaug_student
0.2947 $\pm$ 0.1582 & 0.7739 $\pm$ 0.0843 & 0.9908 $\pm$ 0.0058 & 0.6207 $\pm$ 0.2409 & 0.125 $\pm$ 0.1425 & 


#### No-Dropout+Weakaug Experiments

In [39]:
#experiment_names = ['ConvNeXt_small_Baseline_nodo_weakaug','ConvNeXt_tiny_Baseline_nodo_weakaug','EB0_Baseline_nodo_weakaug','EB1_Baseline_nodo_weakaug','ResNet50_Baseline_nodo_weakaug',
#                    'ConvNeXt_small_KD_nodo_weakaug','ConvNeXt_tiny_KD_nodo_weakaug','EB0_KD_nodo_weakaug','EB1_KD_nodo_weakaug','ResNet50_KD_nodo_weakaug']
#calc_save_metrics(experiment_names,csv_name='model_metrics_nodo_weakaug.csv')

In [40]:
df = pd.read_csv('export/model_metrics_nodo_weakaug.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_Baseline_nodo_weakaug,0.3245,0.0744,0.2737,0.0614,0.2942,0.0626,0.7534,0.0445,0.8087,...,0.4944,0.1472,0.5746,0.3602,0.2125,0.1879,0.2234,0.1498,0.7935,0.0232
3,ConvNeXt_tiny_Baseline_nodo_weakaug,0.357,0.0251,0.3789,0.0394,0.3668,0.0275,0.7396,0.0291,0.8087,...,0.3665,0.1758,0.1693,0.1024,0.2,0.1741,0.174,0.1302,0.7742,0.0204
4,EB0_Baseline_nodo_weakaug,0.2533,0.0799,0.0842,0.0632,0.1226,0.0767,0.6715,0.0426,0.8174,...,0.5529,0.0735,0.0619,0.0762,0.0625,0.0791,0.061,0.075,0.7760,0.0129
5,EB1_Baseline_nodo_weakaug,0.2333,0.2906,0.0316,0.0421,0.0554,0.0732,0.5611,0.0491,0.8783,...,0.4121,0.1582,0.3243,0.0887,0.275,0.192,0.2484,0.0915,0.7539,0.0188
6,ResNet50_Baseline_nodo_weakaug,0.2622,0.1396,0.1474,0.1172,0.1838,0.1256,0.7405,0.0721,0.8087,...,0.5739,0.0697,0.3167,0.1856,0.2,0.1212,0.2355,0.1356,0.7945,0.0142
7,ConvNeXt_small_KD_nodo_weakaug_teacher,0.3467,0.1101,0.3474,0.2321,0.336,0.17,0.8328,0.0917,0.8174,...,0.5933,0.0493,0.1233,0.1041,0.0625,0.0685,0.0769,0.0723,0.8157,0.0151
8,ConvNeXt_small_KD_nodo_weakaug_student,0.3499,0.0411,0.3579,0.0614,0.3526,0.0479,0.7572,0.0434,0.8087,...,0.5343,0.1357,0.3483,0.184,0.3,0.225,0.2856,0.1931,0.8074,0.0227
9,ConvNeXt_tiny_KD_nodo_weakaug_teacher,0.426,0.0426,0.5368,0.1466,0.4705,0.0826,0.8902,0.0598,0.8087,...,0.5919,0.0554,0.175,0.1354,0.0875,0.0848,0.1091,0.0951,0.8240,0.0125
10,ConvNeXt_tiny_KD_nodo_weakaug_student,0.3658,0.0152,0.4842,0.1172,0.4117,0.0546,0.7957,0.1124,0.8087,...,0.4902,0.0662,0.2266,0.1497,0.125,0.1046,0.1549,0.1144,0.7963,0.0094
11,EB0_KD_nodo_weakaug_teacher,0.4969,0.0846,0.3789,0.1021,0.4276,0.0904,0.8719,0.0591,0.913,...,0.5646,0.071,0.1963,0.0687,0.1375,0.1212,0.1484,0.0945,0.8258,0.0135


In [41]:
print_metric_for_latex(df,0,precision_index)

ConvNeXt_small_Baseline_nodo_weakaug
0.3245 $\pm$ 0.0744 & 0.7534 $\pm$ 0.0445 & 0.9558 $\pm$ 0.0077 & 0.4826 $\pm$ 0.0817 & 0.5746 $\pm$ 0.3602 & 


In [42]:
print_metric_for_latex(df,0,recall_index)

ConvNeXt_small_Baseline_nodo_weakaug
0.2737 $\pm$ 0.0614 & 0.8087 $\pm$ 0.0213 & 0.9969 $\pm$ 0.0038 & 0.531 $\pm$ 0.2252 & 0.2125 $\pm$ 0.1879 & 


In [43]:
print_metric_for_latex(df,6,precision_index)

ConvNeXt_small_KD_nodo_weakaug_student
0.3499 $\pm$ 0.0411 & 0.7572 $\pm$ 0.0434 & 0.9848 $\pm$ 0.0082 & 0.5296 $\pm$ 0.0821 & 0.3483 $\pm$ 0.184 & 


In [44]:
print_metric_for_latex(df,6,recall_index)

ConvNeXt_small_KD_nodo_weakaug_student
0.3579 $\pm$ 0.0614 & 0.8087 $\pm$ 0.112 & 0.9892 $\pm$ 0.0038 & 0.5655 $\pm$ 0.2166 & 0.3 $\pm$ 0.225 & 


#### ROI Experiments

In [45]:
#experiment_names = ['ConvNeXt_small_KD_10roi','ConvNeXt_small_KD_30roi','ConvNeXt_small_KD_50roi','ConvNeXt_small_KD_70roi']
#calc_save_metrics(experiment_names,csv_name='model_metrics_roi.csv')

In [46]:
df = pd.read_csv('export/model_metrics_roi.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_KD_10roi_teacher,0.4401,0.0135,0.5263,0.1372,0.4719,0.0682,0.8194,0.0573,0.6609,...,0.537,0.0858,0.26,0.0952,0.075,0.025,0.1093,0.0248,0.8000,0.0257
3,ConvNeXt_small_KD_10roi_student,0.324,0.0526,0.3368,0.0714,0.3297,0.061,0.811,0.0777,0.7739,...,0.4795,0.0586,0.1702,0.0966,0.0875,0.05,0.1149,0.0648,0.7825,0.0150
4,ConvNeXt_small_KD_30roi_teacher,0.4141,0.0449,0.6211,0.154,0.4911,0.0744,0.8218,0.108,0.7304,...,0.5889,0.0644,0.1375,0.1696,0.1375,0.1696,0.1375,0.1696,0.8203,0.0082
5,ConvNeXt_small_KD_30roi_student,0.3595,0.0359,0.4211,0.0666,0.387,0.0467,0.7341,0.0594,0.7826,...,0.5375,0.0911,0.3319,0.193,0.125,0.0791,0.1703,0.1025,0.8009,0.0111
6,ConvNeXt_small_KD_50roi_teacher,0.4616,0.0194,0.7368,0.0744,0.5658,0.0257,0.8721,0.0653,0.7913,...,0.5865,0.0324,0.3156,0.1617,0.2375,0.1275,0.2675,0.1366,0.8378,0.0054
7,ConvNeXt_small_KD_50roi_student,0.3668,0.038,0.4842,0.1306,0.4134,0.0695,0.8222,0.0662,0.7217,...,0.4826,0.0968,0.3874,0.0816,0.3625,0.0468,0.3722,0.0576,0.8028,0.0158
8,ConvNeXt_small_KD_70roi_teacher,0.4147,0.0366,0.5789,0.1451,0.4784,0.0781,0.9426,0.0534,0.8,...,0.5757,0.056,0.3194,0.1047,0.15,0.0637,0.1904,0.0537,0.8267,0.0164
9,ConvNeXt_small_KD_70roi_student,0.3685,0.0289,0.4211,0.0666,0.3921,0.045,0.8032,0.0653,0.7652,...,0.4647,0.1735,0.1292,0.1077,0.1,0.1016,0.1021,0.0919,0.7899,0.0147


#### Extract best models

In [47]:
def print_best_models_per_fold(df):
    min_val_losses = {}
    min_val_loss_epoch = {}
    for fold in range(5):
        min_val_losses[fold] = df[f'fold{fold}_val_loss'].min()
        min_val_loss_epoch[fold] = df[f'fold{fold}_val_loss'].argmin()

    for k,v in min_val_losses.items():
        print(f'Fold {k}:')
        print(f'Epoch: {min_val_loss_epoch[k]}')
        print(f'Min loss: {v}')
        acc = df.loc[min_val_loss_epoch[k],f'fold{k}_val_acc'].item()
        print(f'Accuracy: {acc}')
        print('----------')

In [48]:
df_logs = pd.read_csv('logs/ConvNeXt_small_Baseline_convnext_small_metrics.csv')

In [49]:
print_best_models_per_fold(df_logs)

Fold 0:
Epoch: 8
Min loss: 0.5120167182887205
Accuracy: 0.7880184331797235
----------
Fold 1:
Epoch: 9
Min loss: 0.4800122784030053
Accuracy: 0.7788018433179723
----------
Fold 2:
Epoch: 8
Min loss: 0.5336740906886791
Accuracy: 0.7880184331797235
----------
Fold 3:
Epoch: 6
Min loss: 0.5104876048004572
Accuracy: 0.7880184331797235
----------
Fold 4:
Epoch: 8
Min loss: 0.5342466117050241
Accuracy: 0.783410138248848
----------


-> Modell mit Fold 3 gewählt

In [50]:
df_logs = pd.read_csv('logs/ConvNeXt_small_KD_student_convnext_small_metrics.csv')

In [51]:
print_best_models_per_fold(df_logs)

Fold 0:
Epoch: 15
Min loss: 0.4621245388611121
Accuracy: 0.7972350230414746
----------
Fold 1:
Epoch: 16
Min loss: 0.4763479979906214
Accuracy: 0.8202764976958525
----------
Fold 2:
Epoch: 17
Min loss: 0.4842390069214429
Accuracy: 0.8202764976958525
----------
Fold 3:
Epoch: 18
Min loss: 0.4456024873091878
Accuracy: 0.7972350230414746
----------
Fold 4:
Epoch: 12
Min loss: 0.5106694445632021
Accuracy: 0.8064516129032258
----------


-> Modell mit Fold 1 gewählt

### Loss curves

Convnext_small:

In [52]:
fig, axs = plt.subplots(3,4,figsize=(20,10),dpi=150,sharex=True)
plot_loss_curve('ConvNeXt_small_Baseline','convnext_small',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,0])
plot_loss_curve('ConvNeXt_small_Baseline_nodo','convnext_small',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,1])
plot_loss_curve('ConvNeXt_small_Baseline_weakaug','convnext_small',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,2])
plot_loss_curve('ConvNeXt_small_Baseline_nodo_weakaug','convnext_small',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,3])
plot_loss_curve('ConvNeXt_small_KD','convnext_small',model_type='teacher',plot_type='loss',repeated_runs=5,ax=axs[1,0])
plot_loss_curve('ConvNeXt_small_KD_nodo','convnext_small',model_type='teacher',plot_type='loss',repeated_runs=5,ax=axs[1,1])
plot_loss_curve('ConvNeXt_small_KD_weakaug','convnext_small',model_type='teacher',plot_type='loss',repeated_runs=5,ax=axs[1,2])
plot_loss_curve('ConvNeXt_small_KD_nodo_weakaug','convnext_small',model_type='teacher',plot_type='loss',repeated_runs=5,ax=axs[1,3])
plot_loss_curve('ConvNeXt_small_KD','convnext_small',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[2,0],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_small_KD_nodo','convnext_small',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[2,1],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_small_KD_weakaug','convnext_small',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[2,2],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_small_KD_nodo_weakaug','convnext_small',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[2,3],student_losses=['cons','cls'])
axs[0,0].set_title('Configuration 1')
axs[0,1].set_title('Configuration 2')
axs[0,2].set_title('Configuration 3')
axs[0,3].set_title('Configuration 4')

pad=5
for ax,row in zip(axs[:,0],['a','b','c']):
    ax.annotate(row, xy=(0, 3.9), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                fontsize='xx-large',fontweight='bold', ha='right', va='center')

fig.tight_layout()
fig.subplots_adjust(left=0.05, top=0.95)
plt.savefig(f'export/convnext_small_loss_curves.png')
plt.close()

Convnext_tiny:

In [53]:
fig, axs = plt.subplots(2,4,figsize=(20,10),dpi=150,sharex=True)
plot_loss_curve('ConvNeXt_tiny_Baseline','convnext_tiny',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,0])
plot_loss_curve('ConvNeXt_tiny_Baseline_nodo','convnext_tiny',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,1])
plot_loss_curve('ConvNeXt_tiny_Baseline_weakaug','convnext_tiny',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,2])
plot_loss_curve('ConvNeXt_tiny_Baseline_nodo_weakaug','convnext_tiny',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,3])
plot_loss_curve('ConvNeXt_tiny_KD','convnext_tiny',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,0],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_tiny_KD_nodo','convnext_tiny',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,1],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_tiny_KD_weakaug','convnext_tiny',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,2],student_losses=['cons','cls'])
plot_loss_curve('ConvNeXt_tiny_KD_nodo_weakaug','convnext_tiny',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,3],student_losses=['cons','cls'])
axs[0,0].set_title('Configuration 1')
axs[0,1].set_title('Configuration 2')
axs[0,2].set_title('Configuration 3')
axs[0,3].set_title('Configuration 4')

pad=5
for ax,row in zip(axs[:,0],['a','b']):
    ax.annotate(row, xy=(0, 6.7), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                fontsize='xx-large',fontweight='bold', ha='right', va='center')

fig.tight_layout()
fig.subplots_adjust(left=0.05, top=0.95)
plt.savefig(f'export/convnext_tiny_loss_curves.png')
plt.close()

EB0:

In [54]:
fig, axs = plt.subplots(2,4,figsize=(20,10),dpi=150,sharex=True)
plot_loss_curve('EB0_Baseline','efficientnet_b0',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,0])
plot_loss_curve('EB0_Baseline_nodo','efficientnet_b0',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,1])
plot_loss_curve('EB0_Baseline_weakaug','efficientnet_b0',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,2])
plot_loss_curve('EB0_Baseline_nodo_weakaug','efficientnet_b0',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,3])
plot_loss_curve('EB0_KD','efficientnet_b0',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,0],student_losses=['cons','cls'])
plot_loss_curve('EB0_KD_nodo','efficientnet_b0',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,1],student_losses=['cons','cls'])
plot_loss_curve('EB0_KD_weakaug','efficientnet_b0',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,2],student_losses=['cons','cls'])
plot_loss_curve('EB0_KD_nodo_weakaug','efficientnet_b0',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,3],student_losses=['cons','cls'])
axs[0,0].set_title('Configuration 1')
axs[0,1].set_title('Configuration 2')
axs[0,2].set_title('Configuration 3')
axs[0,3].set_title('Configuration 4')

pad=5
for ax,row in zip(axs[:,0],['a','b']):
    ax.annotate(row, xy=(0, 6.7), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                fontsize='xx-large',fontweight='bold', ha='right', va='center')

fig.tight_layout()
fig.subplots_adjust(left=0.05, top=0.95)
plt.savefig(f'export/eb0_loss_curves.png')
plt.close()

EB1:

In [55]:
fig, axs = plt.subplots(2,4,figsize=(20,10),dpi=150,sharex=True)
plot_loss_curve('EB1_Baseline','efficientnet_b1',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,0])
plot_loss_curve('EB1_Baseline_nodo','efficientnet_b1',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,1])
plot_loss_curve('EB1_Baseline_weakaug','efficientnet_b1',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,2])
plot_loss_curve('EB1_Baseline_nodo_weakaug','efficientnet_b1',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,3])
plot_loss_curve('EB1_KD','efficientnet_b1',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,0],student_losses=['cons','cls'])
plot_loss_curve('EB1_KD_nodo','efficientnet_b1',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,1],student_losses=['cons','cls'])
plot_loss_curve('EB1_KD_weakaug','efficientnet_b1',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,2],student_losses=['cons','cls'])
plot_loss_curve('EB1_KD_nodo_weakaug','efficientnet_b1',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,3],student_losses=['cons','cls'])
axs[0,0].set_title('Configuration 1')
axs[0,1].set_title('Configuration 2')
axs[0,2].set_title('Configuration 3')
axs[0,3].set_title('Configuration 4')

pad=5
for ax,row in zip(axs[:,0],['a','b']):
    ax.annotate(row, xy=(0, 6.7), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                fontsize='xx-large',fontweight='bold', ha='right', va='center')

fig.tight_layout()
fig.subplots_adjust(left=0.05, top=0.95)
plt.savefig(f'export/eb1_loss_curves.png')
plt.close()

Resnet:

In [56]:
fig, axs = plt.subplots(2,4,figsize=(20,10),dpi=150,sharex=True)
plot_loss_curve('ResNet50_Baseline','resnet50',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,0])
plot_loss_curve('ResNet50_Baseline_nodo','resnet50',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,1])
plot_loss_curve('ResNet50_Baseline_weakaug','resnet50',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,2])
plot_loss_curve('ResNet50_Baseline_nodo_weakaug','resnet50',model_type='baseline',plot_type='loss',repeated_runs=5,ax=axs[0,3])
plot_loss_curve('ResNet50_KD','resnet50',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,0],student_losses=['cons','cls'])
plot_loss_curve('ResNet50_KD_nodo','resnet50',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,1],student_losses=['cons','cls'])
plot_loss_curve('ResNet50_KD_weakaug','resnet50',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,2],student_losses=['cons','cls'])
plot_loss_curve('ResNet50_KD_nodo_weakaug','resnet50',model_type='student',plot_type='loss',repeated_runs=5,ax=axs[1,3],student_losses=['cons','cls'])
axs[0,0].set_title('Configuration 1')
axs[0,1].set_title('Configuration 2')
axs[0,2].set_title('Configuration 3')
axs[0,3].set_title('Configuration 4')

pad=5
for ax,row in zip(axs[:,0],['a','b']):
    ax.annotate(row, xy=(0, 6.7), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                fontsize='xx-large',fontweight='bold', ha='right', va='center')

fig.tight_layout()
fig.subplots_adjust(left=0.05, top=0.95)
plt.savefig(f'export/resnet50_loss_curves.png')
plt.close()

#### No ROI (teacher gets raw images)

In [57]:
#experiment_names = ['ConvNeXt_small_KD_noroi','ConvNeXt_tiny_KD_noroi','EB0_KD_noroi','EB1_KD_noroi','ResNet50_KD_noroi']
#calc_save_metrics(experiment_names,csv_name='model_metrics_noroi.csv')

In [58]:
df = pd.read_csv('export/model_metrics_noroi.csv')
df.columns = ['model']+[f'{x}_{y}_{z}' for x,y,z in itertools.product([0,1,2,3,4],['Prec','Rec','F1'],['mean','std'])]+['acc_mean','acc_std']
df.drop(index=[0,1],inplace=True)
df

,model,0_Prec_mean,0_Prec_std,0_Rec_mean,0_Rec_std,0_F1_mean,0_F1_std,1_Prec_mean,1_Prec_std,1_Rec_mean,...,3_F1_mean,3_F1_std,4_Prec_mean,4_Prec_std,4_Rec_mean,4_Rec_std,4_F1_mean,4_F1_std,acc_mean,acc_std
2,ConvNeXt_small_KD_noroi_teacher,0.3524,0.1233,0.3263,0.1263,0.3012,0.0324,0.7905,0.1242,0.4522,...,0.5101,0.1271,0.16,0.32,0.05,0.1,0.0762,0.1524,0.7548,0.0385
3,ConvNeXt_small_KD_noroi_student,0.3221,0.0821,0.3579,0.1263,0.3349,0.0913,0.763,0.0419,0.7652,...,0.5125,0.0582,0.2789,0.1403,0.2,0.1212,0.2279,0.1219,0.7880,0.0140
4,ConvNeXt_tiny_KD_noroi_teacher,0.359,0.021,0.6,0.1396,0.4439,0.0424,0.8548,0.0812,0.287,...,0.5376,0.1065,0.0,0.0,0.0,0.0,0.0,0.0,0.7659,0.0089
5,ConvNeXt_tiny_KD_noroi_student,0.3737,0.0711,0.3053,0.0211,0.3314,0.0245,0.7651,0.0218,0.7565,...,0.4471,0.0833,0.3083,0.0685,0.325,0.1212,0.3133,0.093,0.7825,0.0176
6,EB0_KD_noroi_teacher,0.1,0.2,0.0105,0.0211,0.019,0.0381,0.7957,0.1225,0.6087,...,0.5714,0.0836,0.1483,0.1223,0.1375,0.1649,0.1228,0.1175,0.7650,0.0200
7,EB0_KD_noroi_student,0.2857,0.0888,0.1158,0.0394,0.1643,0.0539,0.6974,0.0382,0.7913,...,0.5516,0.1253,0.0933,0.1218,0.125,0.2201,0.0964,0.1516,0.7862,0.0171
8,EB1_KD_noroi_teacher,0.1444,0.1975,0.0316,0.0421,0.0476,0.0602,0.1445,0.0232,0.8174,...,0.1089,0.136,0.2105,0.0814,0.4375,0.2092,0.251,0.0379,0.2378,0.0873
9,EB1_KD_noroi_student,0.3233,0.102,0.1684,0.1575,0.2062,0.1484,0.7222,0.1043,0.7391,...,0.5033,0.0842,0.3298,0.0575,0.35,0.1159,0.3261,0.0856,0.7788,0.0246
10,ResNet50_KD_noroi_teacher,0.0,0.0,0.0,0.0,0.0,0.0,0.0667,0.1333,0.0087,...,0.1064,0.0991,0.0,0.0,0.0,0.0,0.0,0.0,0.6111,0.0115
11,ResNet50_KD_noroi_student,0.1586,0.1531,0.0526,0.0471,0.0783,0.0706,0.7236,0.1075,0.6348,...,0.5672,0.0247,0.1286,0.1938,0.025,0.0306,0.0396,0.0491,0.7751,0.0068
